In [1]:
import os
from pathlib import Path
from time import strftime

import mlflow
import pandas as pd
import tensorflow as tf

In [2]:
pwd()

'c:\\Main Workspace\\LogAnomalyDetect\\development'

In [3]:
%cd ..

c:\Main Workspace\LogAnomalyDetect


In [6]:
from src.dataset.data_loader import get_dataset, preprocess_and_encode, get_vectorization_layer
from src.utils.common_utils import get_device_strategy
from src.utils.common_utils import set_seed

ModuleNotFoundError: No module named 'dataset'

In [5]:
set_seed()

NameError: name 'set_seed' is not defined

In [8]:
data_path = 'development/dev.gzip'
clean_dev = "development/clean_dev.gzip"

In [9]:
df = pd.read_parquet(data_path)
df.head()

,Log,Target
0,1119803499 2005.06.26 R03-M0-NC-C:J03-U01 200...,normal
1,1119803105 2005.06.26 R04-M0-NE-C:J06-U11 200...,normal
2,1121496169 2005.07.15 R06-M0-N6-C:J03-U01 200...,normal
3,1120968564 2005.07.09 R26-M0-N0-C:J03-U01 200...,normal
4,1120953205 2005.07.09 R27-M0-N7-C:J11-U01 200...,normal


In [10]:
df.shape

(1000, 2)

In [11]:
#preprocess_and_encode(data_path, "development/clean_dev.gzip")

In [12]:
df_dataset = get_dataset(data_path)

In [13]:
for idx in df_dataset.take(3):
    print(idx)

(<tf.Tensor: shape=(2,), dtype=string, numpy=
array([b' 1119803499 2005.06.26 R03-M0-NC-C:J03-U01 2005-06-26-09.31.39.485965 R03-M0-NC-C:J03-U01 RAS KERNEL INFO data store interrupt caused by dcbf.........0\n',
       b' 1119803105 2005.06.26 R04-M0-NE-C:J06-U11 2005-06-26-09.25.05.389021 R04-M0-NE-C:J06-U11 RAS KERNEL INFO program interrupt: illegal instruction......1\n'],
      dtype=object)>, <tf.Tensor: shape=(2,), dtype=string, numpy=array([b'normal', b'normal'], dtype=object)>)
(<tf.Tensor: shape=(2,), dtype=string, numpy=
array([b' 1121496169 2005.07.15 R06-M0-N6-C:J03-U01 2005-07-15-23.42.49.989372 R06-M0-N6-C:J03-U01 RAS KERNEL INFO generating core.1735\n',
       b' 1120968564 2005.07.09 R26-M0-N0-C:J03-U01 2005-07-09-21.09.24.789209 R26-M0-N0-C:J03-U01 RAS KERNEL INFO generating core.16239\n'],
      dtype=object)>, <tf.Tensor: shape=(2,), dtype=string, numpy=array([b'normal', b'normal'], dtype=object)>)
(<tf.Tensor: shape=(2,), dtype=string, numpy=
array([b' 1120953205 2005

In [14]:
clean_dataset = get_dataset(clean_dev)

In [15]:
for idx in clean_dataset.take(2):
    print(idx)

(<tf.Tensor: shape=(2,), dtype=string, numpy=
array([b'rmnccju rmnccju ras kernel info data store interrupt caused dcbf',
       b'rmnecju rmnecju ras kernel info program interrupt illegal instruction'],
      dtype=object)>, <tf.Tensor: shape=(2,), dtype=int16, numpy=array([0, 0], dtype=int16)>)
(<tf.Tensor: shape=(2,), dtype=string, numpy=
array([b'rmncju rmncju ras kernel info generating core',
       b'rmncju rmncju ras kernel info generating core'], dtype=object)>, <tf.Tensor: shape=(2,), dtype=int16, numpy=array([0, 0], dtype=int16)>)


In [16]:
token_layer, vocab_s = get_vectorization_layer(clean_dataset)

In [17]:
vocab_s

420

In [18]:
a = token_layer("k e r n e l i n f o")

In [19]:
a

<tf.Tensor: shape=(20,), dtype=int64, numpy=
array([  1, 151, 291, 310, 151, 184,   1, 310, 162,   1,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0], dtype=int64)>

In [20]:
S = get_device_strategy()
S.num_replicas_in_sync

1

In [21]:
from src.models.model_loader import ModelLoader

In [22]:
loader = ModelLoader()
test_model_loader = loader.get_model("cnn")

In [39]:
test_model = test_model_loader(embedding_vocab= vocab_s, embedding_dim= 5,
                        vectorization_layer=token_layer)

In [44]:
loss = tf.keras.losses.BinaryCrossentropy()
optim = tf.keras.optimizers.Adadelta(learning_rate=0.01)
tensorb = tf.keras.callbacks.TensorBoard()
f1_score = tf.keras.metrics.F1Score()

In [41]:
test_model.compile(loss=loss, optimizer=optim)

In [42]:
test_model.fit(clean_dataset,epochs=20, validation_data= clean_dataset)

Epoch 1/20


500/500 [==============================] - 6s 9ms/step - loss: 0.6863 - val_loss: 0.6790
Epoch 2/20
500/500 [==============================] - 4s 7ms/step - loss: 0.6715 - val_loss: 0.6627
Epoch 3/20
500/500 [==============================] - 3s 7ms/step - loss: 0.6536 - val_loss: 0.6418
Epoch 4/20
500/500 [==============================] - 3s 7ms/step - loss: 0.6304 - val_loss: 0.6179
Epoch 5/20
500/500 [==============================] - 3s 7ms/step - loss: 0.6084 - val_loss: 0.5926
Epoch 6/20
500/500 [==============================] - 4s 7ms/step - loss: 0.5784 - val_loss: 0.5634
Epoch 7/20
500/500 [==============================] - 4s 7ms/step - loss: 0.5538 - val_loss: 0.5329
Epoch 8/20
500/500 [==============================] - 3s 6ms/step - loss: 0.5198 - val_loss: 0.4995
Epoch 9/20
500/500 [==============================] - 3s 6ms/step - loss: 0.4875 - val_loss: 0.4642
Epoch 10/20
500/500 [==============================] - 3s 7ms/step - loss: 0.4586 - val_loss: 0.4288
Epoch 11/2

In [45]:
with S.scope():
    token_layer, vocab_s = get_vectorization_layer(clean_dataset)
    new_test_model = test_model_loader(embedding_vocab= vocab_s, embedding_dim= 5,
                        vectorization_layer=token_layer)
    new_test_model.compile(loss=loss, optimizer=optim)
    new_test_model.fit(clean_dataset,epochs=2, validation_data= clean_dataset)

Epoch 1/2
500/500 [==============================] - 10s 15ms/step - loss: 0.6919 - val_loss: 0.6830
Epoch 2/2
500/500 [==============================] - 4s 8ms/step - loss: 0.6743 - val_loss: 0.6653


In [ ]:
test_model.fit()

In [ ]:
"""def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return tokenizer(text), label
final_dataset = dataset.map(vectorize_text)
for sample in final_dataset.take(1):
    print(sample)"""

(<tf.Tensor: shape=(2, 20), dtype=int64, numpy=
array([[  2,   2,   3,   4,   5,  29,  30, 271,  28,  23,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0],
       [  2,   2,   3,   4,   5,   7,   6,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0]])>, <tf.Tensor: shape=(2,), dtype=float32, numpy=array([0., 0.], dtype=float32)>)
